# Fast 8-point DCT & IDCT Algorithms
This repo contains an implementation and derivation of two algorithms for fast DCT/IDCT computations. The second one provides an effective fusion of DCT/IDCT with a prior or subsequent quantization process if there is any. The resulted **speed up**  can be **up to 7 times** in comparison with a naive approach.

![execution time](./assets/execution_time.png)

## Introduction
DCT (DCT-II) describes the input signal in terms of frequences and represents it with the cosine basis functions in real domain. The orthogonal transformation of N-length input $x_n$ is defined by

$$
y_n = \sum\limits_{k=0}^{N-1} c_k x_k \cos\left(\frac{\pi n(2k + 1)}{2N}\right)
$$
where $c_0 = \frac{1}{\sqrt{N}}$ and $c_k = \sqrt{\frac{2}{N}}$ for $1 \leq k \leq N - 1$ are normalization coefficients to ensure the orthogonality of transformation.

## Motivation
DCT is a widely used transformation technique in signal processing and its 8-point version is an important step in many data compression applications, therefore a variety of algorithms for fast DCT were proposed and two of them are presented here.

## Statement
The 8-point orthogonal DCT & IDCT transformations can be computed with **13 multiplications** and **29 additions**. In comparisson a naive approach requires 64 multiplications and 56 additions. 
NOTE: if we ommit the orthogonality requirement then a scaled transformation by factor of $\sqrt{2}$ can be performed with only **11 multiplications**.

## Preliminaries

A 8-point orthogonal DCT matrix is defined by
$$
W = \frac{1}{2}\left[\begin{matrix*}[r]W_{4} & W_{4} & W_{4} & W_{4} & W_{4} & W_{4} & W_{4} & W_{4}\\W_{1} & W_{3} & W_{5} & W_{7} & - W_{7} & - W_{5} & - W_{3} & - W_{1}\\W_{2} & W_{6} & - W_{6} & - W_{2} & - W_{2} & - W_{6} & W_{6} & W_{2}\\W_{3} & - W_{7} & - W_{1} & - W_{5} & W_{5} & W_{1} & W_{7} & - W_{3}\\W_{4} & - W_{4} & - W_{4} & W_{4} & W_{4} & - W_{4} & - W_{4} & W_{4}\\W_{5} & - W_{1} & W_{7} & W_{3} & - W_{3} & - W_{7} & W_{1} & - W_{5}\\W_{6} & - W_{2} & W_{2} & - W_{6} & - W_{6} & W_{2} & - W_{2} & W_{6}\\W_{7} & - W_{5} & W_{3} & - W_{1} & W_{1} & - W_{3} & W_{5} & - W_{7}\end{matrix*}\right]
$$
where $W_k = \cos\left(\frac{\pi k}{16}\right)$ and $\frac{W_0}{\sqrt{2}} = W_4$.

Both derivations will be based on the following equalities (sum-to-product identities):
$$
\displaylines{
W_{k - n} + W_{k + n} = 2W_k W_n \\
W_{k - n} - W_{16 - (k + n)} = 2W_{k} W_{n}
}
$$

The first step is to split even and old samples by averaging columns $c_{k}$ and $c_{8 - k}$ (so-called butterfly transformation):

$$
A_3^{-1} = \left[\begin{matrix*}[r]\frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{2}\\0 & \frac{1}{2} & 0 & 0 & 0 & 0 & \frac{1}{2} & 0\\0 & 0 & \frac{1}{2} & 0 & 0 & \frac{1}{2} & 0 & 0\\0 & 0 & 0 & \frac{1}{2} & \frac{1}{2} & 0 & 0 & 0\\0 & 0 & 0 & \frac{1}{2} & - \frac{1}{2} & 0 & 0 & 0\\0 & 0 & \frac{1}{2} & 0 & 0 & - \frac{1}{2} & 0 & 0\\0 & \frac{1}{2} & 0 & 0 & 0 & 0 & - \frac{1}{2} & 0\\\frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0 & - \frac{1}{2}\end{matrix*}\right]
%
%
A_3 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\0 & 1 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 1 & 0 & 0 & 1 & 0 & 0\\0 & 0 & 0 & 1 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 1 & -1 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & -1 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & -1 & 0\\1 & 0 & 0 & 0 & 0 & 0 & 0 & -1\end{matrix*}\right]
$$

$$ 
W A_3^{-1} = \frac{1}{2}
\left[\begin{matrix*}[r]W_{4} & W_{4} & W_{4} & W_{4} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{7} & W_{5} & W_{3} & W_{1}\\W_{2} & W_{6} & - W_{6} & - W_{2} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & - W_{5} & - W_{1} & - W_{7} & W_{3}\\W_{4} & - W_{4} & - W_{4} & W_{4} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{3} & W_{7} & - W_{1} & W_{5}\\W_{6} & - W_{2} & W_{2} & - W_{6} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & - W_{1} & W_{3} & - W_{5} & W_{7}\end{matrix*}\right]
$$

Continue by splitting columns 1-4 and averaging $c_k$ and $c_{4 - k}$:

$$
A_2^{-1} = \left[\begin{matrix*}[r]\frac{1}{2} & 0 & 0 & \frac{1}{2} & 0 & 0 & 0 & 0\\0 & \frac{1}{2} & \frac{1}{2} & 0 & 0 & 0 & 0 & 0\\0 & \frac{1}{2} & - \frac{1}{2} & 0 & 0 & 0 & 0 & 0\\\frac{1}{2} & 0 & 0 & - \frac{1}{2} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
%
A_2 = \left[\begin{matrix*}[r]1 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 1 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 1 & -1 & 0 & 0 & 0 & 0 & 0\\1 & 0 & 0 & -1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$

$$
W A_3^{-1}A_2^{-1} = \frac{1}{2}\left[\begin{matrix*}[r]W_{4} & W_{4} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{7} & W_{5} & W_{3} & W_{1}\\0 & 0 & W_{6} & W_{2} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -W_{5} & - W_{1} & - W_{7} & W_{3}\\W_{4} & - W_{4} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{3} & W_{7} & - W_{1} & W_{5}\\0 & 0 & - W_{2} & W_{6} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -W_{1} & W_{3} & - W_{5} & W_{7}\end{matrix*}\right]
$$

Split columns $c_1$ and $c_2$:

$$
A_1^{-1} = \left[\begin{matrix*}[r]\frac{1}{2} & \frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0\\\frac{1}{2} & - \frac{1}{2} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
%
A_1 = \left[\begin{matrix*}[r]1 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\1 & -1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$

$$
A = W A_3^{-1}A_2^{-1}A_1^{-1} = \frac{1}{2}\left[\begin{matrix*}[r]W_{4} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{7} & W_{5} & W_{3} & W_{1}\\0 & 0 & W_{6} & W_{2} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -W_{5} & - W_{1} & - W_{7} & W_{3}\\0 & W_{4} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{3} & W_{7} & - W_{1} & W_{5}\\0 & 0 & - W_{2} & W_{6} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & -W_{1} & W_{3} & - W_{5} & W_{7}\end{matrix*}\right]
$$

## Algorithm 1
### Derivation

To this end we used a variation of Cooley–Tukey algorithm. Now permute rows:

$$
P^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
%
P = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$

$$
T = P^{-1} A = \frac{1}{2}\left[\begin{matrix*}[r] W_{4} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & W_{4} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & - W_{2} & W_{6} & 0 & 0 & 0 & 0\\0 & 0 & W_{6} & W_{2} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{7} & W_{5} & W_{3} & W_{1}\\0 & 0 & 0 & 0 & -W_{5} & - W_{1} & - W_{7} & W_{3}\\0 & 0 & 0 & 0 & W_{3} & W_{7} & -W_{1} & W_{5}\\0 & 0 & 0 & 0 & - W_{1} & W_{3} & - W_{5} & W_{7}\end{matrix*}\right]
$$

The only idea left to notice is the presence of rotation transformations in columns 3, 4; 6, 7 and 5, 8 (see entries $t_{33}, t_{34}, t_{43}, t_{44}$ and $t_{55}, t_{58}, t_{85}, t_{88}$ as well as $t_{56}, t_{57}, t_{86}, t_{87}$) as

$$
\displaylines{
W_k = \cos\left(\alpha\right) \\
W_{8 - k} = \sin\left(\alpha\right)
}
$$

We need to scale the first two columns by a factor of $\frac{1}{W_4}$ and use the inverse rotations for columns 3, 4; 6, 7 and 5, 8. This is done by the following transformation:

$$
B_3^{-1} = 2\left[\begin{matrix*}[r]\frac{1}{W_{4}} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & \frac{1}{W_{4}} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & - W_{2} & W_{6} & 0 & 0 & 0 & 0\\0 & 0 & W_{6} & W_{2} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{7} & 0 & 0 & - W_{1}\\0 & 0 & 0 & 0 & 0 & W_{5} & W_{3} & 0\\0 & 0 & 0 & 0 & 0 & W_{3} & -W_{5} & 0\\0 & 0 & 0 & 0 & W_{1} & 0 & 0 & W_{7}\end{matrix*}\right]
$$

$$
B_3 = \frac{1}{2}\left[\begin{matrix*}[r]W_{4} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & W_{4} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & - W_{2} & W_{6} & 0 & 0 & 0 & 0\\0 & 0 & W_{6} & W_{2} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{7} & 0 & 0 & W_{1}\\0 & 0 & 0 & 0 & 0 & W_{5} & W_{3} & 0\\0 & 0 & 0 & 0 & 0 & W_{3} & - W_{5} & 0\\0 & 0 & 0 & 0 & - W_{1} & 0 & 0 & W_{7}\end{matrix*}\right]
$$

$$
P^{-1}AB_3^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0\\0 & 0 & 0 & 1 & 0\\0 & 0 & 0 & 0 & C\end{matrix*}\right]
$$
where

$$
C = \left[\begin{matrix*}[r]1 & 1 & 0 & 0 \\ W_{1} W_{3} - W_{5} W_{7} & - W_{1} W_{5} - W_{3} W_{7} & - W_{1} W_{3} + W_{5} W_{7} & W_{1} W_{5} + W_{3} W_{7}\\ W_{1} W_{5} + W_{3} W_{7} & - W_{1} W_{3} + W_{5} W_{7} & W_{1} W_{5} + W_{3} W_{7} & - W_{1} W_{3} + W_{5} W_{7}\\0 & 0 & 1 & 1\end{matrix*}\right]
$$

With identities above this results in

$$
C = \left[\begin{matrix*}[r]1 & 1 & 0 & 0\\W_{4} & - W_{4} & - W_{4} & W_{4}\\W_{4} & - W_{4} & W_{4} & - W_{4}\\0 & 0 & 1 & 1\end{matrix*}\right]
$$

Butterfly transformations for the first two and the last two columns:

$$
B_2^{-1} = \left[\begin{matrix*}[r]\frac{1}{2} & \frac{1}{2} & 0 & 0\\\frac{1}{2} & - \frac{1}{2} & 0 & 0\\0 & 0 & \frac{1}{2} & \frac{1}{2}\\0 & 0 & - \frac{1}{2} & \frac{1}{2}\end{matrix*}\right]
%
B_2 = \left[\begin{matrix*}[r]1 & 1 & 0 & 0\\1 & -1 & 0 & 0\\0 & 0 & 1 & -1\\0 & 0 & 1 & 1\end{matrix*}\right]
$$

$$
CB_2^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0\\0 & W_{4} & - W_{4} & 0\\0 & W_{4} & W_{4} & 0\\0 & 0 & 0 & 1\end{matrix*}\right]
$$

And the final step

$$
B_1^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0\\0 & \frac{1}{2} & \frac{1}{2} & 0\\0 & - \frac{1}{2} & \frac{1}{2} & 0\\0 & 0 & 0 & 1\end{matrix*}\right]
%
B_1 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0\\0 & 1 & -1 & 0\\0 & 1 & 1 & 0\\0 & 0 & 0 & 1\end{matrix*}\right]
$$

$$
D = CB_2^{-1}B_1^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0\\0 & W_{4} & 0 & 0\\0 & 0 & W_{4} & 0\\0 & 0 & 0 & 1\end{matrix*}\right]
$$

### Counting operations
Note that rotation can be done with $3$ additions (instead of $2$) and $3$ multiplications (instead of $4$):

$$
\displaylines{
y_1 = a x_1 + b x_2 = a (x_1 + x_2) + x_2 (b - a) \\
y_2 = -b x_1 + a x_2 = a (x_1 + x_2) - x_1 (b + a)
}
$$

The multiplications occur only in D and $B_3$:
* 2 multiplications in D
* 11 multiplications in $B_3$: 2 for scaling and 9 for rotations (each of them requires 3 multiplications).

Therefore, **13 multiplications**.

Now the additions counting:
* 2 in B1
* 4 in B2
* 9 in B3 (3 for each rotation)
* 2 in A1
* 4 in A2
* 8 in A3

Correspondingly, **29 additions**.

### Final Decomposition
**NOTE**: here $W_k = \mathbf{0.5} \cos\left(\frac{\pi k}{16}\right)$

There is an option to reduce the number of matricies by one to obtain the final decomposition:

$$
W = P B_1 B_2 A_1 A_2 A_3
$$
where
$$
A_3 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\0 & 1 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 1 & 0 & 0 & 1 & 0 & 0\\0 & 0 & 0 & 1 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 1 & -1 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & -1 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & -1 & 0\\1 & 0 & 0 & 0 & 0 & 0 & 0 & -1\end{matrix*}\right]
$$
$$
A_2 = \left[\begin{matrix*}[r]1 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 1 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 1 & -1 & 0 & 0 & 0 & 0 & 0\\1 & 0 & 0 & -1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{7} & 0 & 0 & W_{1}\\0 & 0 & 0 & 0 & 0 & W_{5} & W_{3} & 0\\0 & 0 & 0 & 0 & 0 & W_{3} & - W_{5} & 0\\0 & 0 & 0 & 0 & - W_{1} & 0 & 0 & W_{7}\end{matrix*}\right]
$$
$$
A_1 = \left[\begin{matrix*}[r]1 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\1 & -1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & - W_{2} & W_{6} & 0 & 0 & 0 & 0\\0 & 0 & W_{6} & W_{2} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 1 & -1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & -1\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 1\end{matrix*}\right]
$$
$$
B_2 = \left[\begin{matrix*}[r]W_{4} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & W_{4} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & -1 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 1 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$
$$
B_1 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 2W_{4} & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 2W_{4} & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$
$$
P = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$

### Inverse Transformation
W is orthogonal, therefore

$$
W^{-1} = W^T = A_3^T A_2^T A_1^T B_2^T B_1^T P^T
$$

## Algorithm 2
### Derivation

The derivation will continue a preliminarie part. First of all, scale matrix A:

$$
S = \frac{1}{4}\left[\begin{matrix*}\frac{1}{W_{4}} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & \frac{1}{W_{1}} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & \frac{1}{W_{2}} & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & \frac{1}{W_{3}} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & \frac{1}{W_{4}} & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & \frac{1}{W_{5}} & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{W_{6}} & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{W_{7}}\end{matrix*}\right]
$$

$$
P^{-1} S^{-1} A = 2\left[\begin{matrix*}[r]W_{4}^{2} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & W_{4}^{2} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & - W_{2} W_{6} & W_{6}^{2} & 0 & 0 & 0 & 0\\0 & 0 & W_{2} W_{6} & W_{2}^{2} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{1} W_{7} & W_{1} W_{5} & W_{1} W_{3} & W_{1}^{2}\\0 & 0 & 0 & 0 & - W_{3} W_{5} & - W_{1} W_{3} & - W_{3} W_{7} & W_{3}^{2}\\0 & 0 & 0 & 0 & W_{3} W_{5} & W_{5} W_{7} & - W_{1} W_{5} & W_{5}^{2}\\0 & 0 & 0 & 0 & - W_{1} W_{7} & W_{3} W_{7} & - W_{5} W_{7} & W_{7}^{2}\end{matrix*}\right]
$$

For simplicity split the above matrix in two:

$$
C_1 = 2\left[\begin{matrix*}[r]W_{4}^{2} & 0 & 0 & 0\\0 & W_{4}^{2} & 0 & 0\\0 & 0 & - W_{2} W_{6} & W_{6}^{2}\\0 & 0 & W_{2} W_{6} & W_{2}^{2}\end{matrix*}\right]
\,\,\,\,
C_2 = 2\left[\begin{matrix*}[r]W_{1} W_{7} & W_{1} W_{5} & W_{1} W_{3} & W_{1}^{2}\\- W_{3} W_{5} & - W_{1} W_{3} & - W_{3} W_{7} & W_{3}^{2}\\W_{3} W_{5} & W_{5} W_{7} & - W_{1} W_{5} & W_{5}^{2}\\- W_{1} W_{7} & W_{3} W_{7} & - W_{5} W_{7} & W_{7}^{2}\end{matrix*}\right]
$$

With identities stated in preliminaries these equalities can be rewriten into

$$
C_1 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0\\0 & 1 & 0 & 0\\0 & 0 & - W_{4} & 1 - W_{4}\\0 & 0 & W_{4} & 1 + W_{4}\end{matrix*}\right]
\,\,\,\,
C_2 = \left[\begin{matrix*}[r]W_{6} & W_{4} + W_{6} & W_{2} + W_{4} & 1 + W_{2}\\- W_{2} & - W_{2} - W_{4} & -W_{4} + W_{6} & 1+ W_{6}\\W_{2} & W_{2} - W_{4} & - W_{4} - W_{6} & 1 - W_{6}\\- W_{6} & W_{4} - W_{6} & - W_{2} + W_{4} & 1 - W_{2}\end{matrix*}\right]
$$

For further transformation subtract the third column of matrix $C_1$ from the fourth and $k$-th column of matrix $C_2$ from $k+1$-th: 

$$
B_3^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & -1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & -1 & 1 & -1\\0 & 0 & 0 & 0 & 0 & 1 & -1 & 1\\0 & 0 & 0 & 0 & 0 & 0 & 1 & -1\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
\,\,\,\,
B_3 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 1 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 1\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$

$$
P^{-1} S^{-1} A B_3^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & - W_{4} & 1 & 0 & 0 & 0 & 0\\0 & 0 & W_{4} & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{6} & W_{4} & W_{2} & 1\\0 & 0 & 0 & 0 & - W_{2} & - W_{4} & W_{6} & 1\\0 & 0 & 0 & 0 & W_{2} & - W_{4} & - W_{6} & 1\\0 & 0 & 0 & 0 & - W_{6} & W_{4} & - W_{2} & 1\end{matrix*}\right]
$$

Apply scaling to the 3rd and the 6th columns and rotation to the 5th and the 7th columns:

$$
B_2^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & \frac{1}{W_{4}} & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{6} & 0 & - W_{2} & 0\\0 & 0 & 0 & 0 & 0 & \frac{1}{W_{4}} & 0 & 0\\0 & 0 & 0 & 0 & W_{2} & 0 & W_{6} & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
%
B_2 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & W_{4} & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{6} & 0 & W_{2} & 0\\0 & 0 & 0 & 0 & 0 & W_{4} & 0 & 0\\0 & 0 & 0 & 0 & - W_{2} & 0 & W_{6} & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$

$$
P^{-1} S^{-1} A B_3^{-1} B_2^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & -1 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 1 & 0 & 1\\0 & 0 & 0 & 0 & 0 & -1 & 1 & 1\\0 & 0 & 0 & 0 & 0 & -1 & -1 & 1\\0 & 0 & 0 & 0 & -1 & 1 & 0 & 1\end{matrix*}\right]
$$

Finally, use the butterfly transformation for 3rd and 4th columns as well as for 6th and 8th columns:

$$
B_1^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & - \frac{1}{2} & \frac{1}{2} & 0 & 0 & 0 & 0\\0 & 0 & \frac{1}{2} & \frac{1}{2} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & \frac{1}{2} & 0 & - \frac{1}{2}\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 0 & 0 & 0 & \frac{1}{2} & 0 & \frac{1}{2}\end{matrix*}\right]
%
B_1 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & -1 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 1\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 0 & 0 & 0 & -1 & 0 & 1\end{matrix*}\right]
$$

$$
C = P^{-1} S^{-1} A B_3^{-1} B_2^{-1} B_1^{-1} = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 1\\0 & 0 & 0 & 0 & 0 & 0 & -1 & 1\\0 & 0 & 0 & 0 & -1 & 1 & 0 & 0\end{matrix*}\right]
$$

### Counting operations

The multiplications occur only in $B_2$ and $S$:
* 8 multiplications in S
* 5 multiplications in $B_2$: 2 for scaling and 3 for one rotation

Therefore, **13 multiplications**.

Now the additions counting:
* 4 in C
* 4 in B1
* 3 in B2 (for rotation)
* 4 in B3
* 2 in A1
* 4 in A2
* 8 in A3

Correspondingly, **29 additions**.

However, compared to the non-scaled algorithm scaling matrix $S$ can be incorporated into output (for DCT) or input (IDCT) signal quantization if there is any and thus only **5 multiplications** is required.

### Final Decomposition
**NOTE**: here $W_k = \cos\left(\frac{\pi k}{16}\right)$

There is an option to reduce the number of matricies by one to obtain the final decomposition:

$$
W = S P B_1 B_2 A_1 A_2 A_3
$$

where

$$
A_3 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 1\\0 & 1 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 1 & 0 & 0 & 1 & 0 & 0\\0 & 0 & 0 & 1 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 1 & -1 & 0 & 0 & 0\\0 & 0 & 1 & 0 & 0 & -1 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & -1 & 0\\1 & 0 & 0 & 0 & 0 & 0 & 0 & -1\end{matrix*}\right]
$$

$$
A_2 = \left[\begin{matrix*}[r]1 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 1 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 1 & -1 & 0 & 0 & 0 & 0 & 0\\1 & 0 & 0 & -1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 1 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 1\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$

$$
A_1 = \left[\begin{matrix*}[r]1 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\1 & -1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & W_{6} & 0 & W_{2} & 0\\0 & 0 & 0 & 0 & 0 & W_{4} & 0 & 0\\0 & 0 & 0 & 0 & - W_{2} & 0 & W_{6} & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$

$$
B_2 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & W_{4} & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 1\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 0 & 0 & 0 & -1 & 0 & 1\end{matrix*}\right]
$$

$$
B_1 = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & -1 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 1 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 1 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 1\\0 & 0 & 0 & 0 & 0 & 0 & -1 & 1\\0 & 0 & 0 & 0 & -1 & 1 & 0 & 0\end{matrix*}\right]
$$

$$
P = \left[\begin{matrix*}[r]1 & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 1 & 0 & 0 & 0\\0 & 0 & 0 & 1 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 1 & 0 & 0\\0 & 1 & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 1 & 0\\0 & 0 & 1 & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & 1\end{matrix*}\right]
$$

$$
S = \frac{1}{4}\left[\begin{matrix}\frac{1}{W_{4}} & 0 & 0 & 0 & 0 & 0 & 0 & 0\\0 & \frac{1}{W_{1}} & 0 & 0 & 0 & 0 & 0 & 0\\0 & 0 & \frac{1}{W_{2}} & 0 & 0 & 0 & 0 & 0\\0 & 0 & 0 & \frac{1}{W_{3}} & 0 & 0 & 0 & 0\\0 & 0 & 0 & 0 & \frac{1}{W_{4}} & 0 & 0 & 0\\0 & 0 & 0 & 0 & 0 & \frac{1}{W_{5}} & 0 & 0\\0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{W_{6}} & 0\\0 & 0 & 0 & 0 & 0 & 0 & 0 & \frac{1}{W_{7}}\end{matrix}\right]
$$

### Inverse Transformation
W is orthogonal, therefore

$$
W^{-1} = W^T = A_3^T A_2^T A_1^T B_2^T B_1^T P^T S^T
$$